# Protótipo de otimização do Lakehouse

In [1]:
!pip install --quiet azure-storage-blob delta-spark

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.9.0 which is incompatible.


In [2]:
from delta.tables import *
from azure.storage.blob import BlobServiceClient, BlobPrefix

In [3]:
#
# Nome da aplicação Spark
#
APP_NAME="OptimizeLakeHouse"

In [4]:
%run StartSpark.ipynb
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "False")

In [5]:
delta_path, _ = paths("stocks", "StockData", "bronze")
url = f"DefaultEndpointsProtocol=https;AccountName={STORAGE_ACCOUNT};AccountKey={STORAGE_KEY};EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(conn_str=url, container_name=BLOB_CONTAINER, blob_name=f'{LAKEHOUSE_PATH}/bronze/StockData_stocks')
container_client = blob_service_client.get_container_client(container="nemesys-stream-101")


In [6]:
def break_path(path:str):
    tokens = path.split("/")
    tier = tokens[1]
    tokens = tokens[2].split("_")
    db = tokens[0]
    table = tokens[1]
    return tier, db, table
    
def findTables(path:str="lakehouse/"):
    tables = []
    
    for blob_tier in container_client.walk_blobs(name_starts_with=path):
        for blob_table in container_client.walk_blobs(name_starts_with=blob_tier.name):
            if "checkpoint" not in blob_table.name:
                tier, db, table = break_path(blob_table.name)
                tables.append((tier, db, table))
    return tables

tables = findTables()

In [ ]:
for tier, db, table in tables:
    delta_path, _ = paths(table, db, tier)
    print(f"Otimizando: {delta_path}")
    deltaTable = DeltaTable.forPath(spark, delta_path)
    deltaTable.optimize().executeCompaction()
    deltaTable.vacuum(4)

Otimizando: abfss://nemesys-stream-101@nemesysdemo.dfs.core.windows.net/lakehouse/bronze/StockData_stocks
